In [35]:
import pandas as pd
import numpy as np
import folium
import osmnx as ox
import networkx as nx
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# High-speed spatial indexing check
try:
    import sklearn
    print("Scikit-learn is ready for fast node lookups.")
except ImportError:
    print("Warning: scikit-learn not found. Install with %pip install scikit-learn")

Scikit-learn is ready for fast node lookups.


In [36]:
city = "London"
dtf = pd.read_csv("data_stores.csv")

# Filter for the city and take 100 samples
dtf = dtf[dtf["City"] == city].copy()
dtf = dtf.iloc[:100].reset_index(drop=True)

# Clean columns and add ID
dtf = dtf[["City", "Street Address", "Latitude", "Longitude"]]
dtf = dtf.reset_index().rename(columns={"index": "ID", "Latitude": "y", "Longitude": "x"})

# Define starting point (Store 0)
start_coords = dtf[dtf['ID'] == 0][["y", "x"]].values[0]
print(f"Total stores: {len(dtf)}")
print(f"Starting point: {start_coords}")
dtf.head()

Total stores: 100
Starting point: [ 42.99 -81.26]


,ID,City,Street Address,y,x
0,0,London,265 Wharncliffe Rd North,42.99,-81.26
1,1,London,"1105 Wellington Rd., Store No. 122",42.93,-81.22
2,2,London,631 Commissioners Road East,42.96,-81.23
3,3,London,1442 Fanshawe Park Road,43.01,-81.34
4,4,London,"580 Fanshawe Park Road East, Unit 1, White Oak...",43.03,-81.26


In [37]:
# Download drive network within 10km of start
G = ox.graph_from_point(tuple(start_coords), dist=10000, network_type="drive")
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
print("Road graph downloaded and travel times added.")

Road graph downloaded and travel times added.


In [38]:
# Vectorized search for all nodes at once (X=Longitude, Y=Latitude)
dtf["node"] = ox.distance.nearest_nodes(G, dtf["x"], dtf["y"])

# Keep only unique road nodes (removes stores that share the same intersection)
dtf_unique = dtf.drop_duplicates("node", keep="first").reset_index(drop=True)

print(f"Mapped {len(dtf)} stores to {len(dtf_unique)} unique road intersections.")
dtf_unique.head()

Mapped 100 stores to 19 unique road intersections.


,ID,City,Street Address,y,x,node
0,0,London,265 Wharncliffe Rd North,42.99,-81.26,289796441
1,1,London,"1105 Wellington Rd., Store No. 122",42.93,-81.22,324811692
2,2,London,631 Commissioners Road East,42.96,-81.23,11756886360
3,3,London,1442 Fanshawe Park Road,43.01,-81.34,252974916
4,4,London,"580 Fanshawe Park Road East, Unit 1, White Oak...",43.03,-81.26,302754873


In [39]:
nodes = dtf_unique["node"].tolist()
n = len(nodes)
dist_matrix = np.zeros((n, n))

print("Calculating 100x100 road distance matrix...")
for i, source in enumerate(nodes):
    # Dijkstra path lengths from 'source' to ALL other nodes in one pass
    lengths = nx.single_source_dijkstra_path_length(G, source, weight='travel_time')
    for j, target in enumerate(nodes):
        # Fill matrix; use a large number if unreachable (for TSP solver)
        dist_matrix[i, j] = int(lengths.get(target, 999999))

print("Matrix complete.")

Calculating 100x100 road distance matrix...
Matrix complete.


In [40]:
def solve_tsp(dist_matrix):
    manager = pywrapcp.RoutingIndexManager(len(dist_matrix), 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        return dist_matrix[manager.IndexToNode(from_index)][manager.IndexToNode(to_index)]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        index, route = routing.Start(0), []
        while not routing.IsEnd(index):
            route.append(manager.IndexToNode(index))
            index = solution.Value(routing.NextVar(index))
        return route

optimized_indices = solve_tsp(dist_matrix)
# Map back to stop nodes and close the loop
optimized_route_nodes = [nodes[i] for i in optimized_indices]
optimized_route_nodes.append(optimized_route_nodes[0])
print(f"Optimal visiting sequence calculated for {len(optimized_indices)} stops.")

Optimal visiting sequence calculated for 19 stops.


In [41]:
m = folium.Map(location=start_coords, zoom_start=13, tiles="cartodbpositron")

# Draw the actual road segments between stops
for i in range(len(optimized_route_nodes) - 1):
    try:
        path = nx.shortest_path(G, optimized_route_nodes[i], optimized_route_nodes[i+1], weight='travel_time')
        path_coords = [[G.nodes[node]['y'], G.nodes[node]['x']] for node in path]
        folium.PolyLine(path_coords, color="#2E86C1", weight=3, opacity=0.8).add_to(m)
    except:
        continue # Skip if no path exists

# Add Markers for stores
for idx, row in dtf_unique.iterrows():
    folium.CircleMarker([row['y'], row['x']], radius=4, color='red' if row['ID']==0 else 'black', fill=True).add_to(m)

m.save("route_map.html")
m

In [51]:
from folium.plugins import TimestampedGeoJson, AntPath
import datetime

# 1. Flatten all road segments into one long list of coordinates
full_path_coords = []
cumulative_time = 0
features = []

# Starting time
start_time = datetime.datetime(2026, 1, 2, 8, 0, 0) # 8:00 AM Today

for path in lst_paths:
    # Get the time for this segment (sum of travel times on edges)
    # If travel_time isn't available, we assume 1 second per node for the animation
    segment_nodes = list(path)
    for i in range(len(segment_nodes) - 1):
        u, v = segment_nodes[i], segment_nodes[i+1]
        
        # Get coordinates
        point = [G.nodes[u]['x'], G.nodes[u]['y']]
        
        # Get travel time (weight) for this specific edge
        edge_data = G.get_edge_data(u, v)
        # Handle cases with multiple edges between nodes
        if isinstance(edge_data, dict) and 0 in edge_data:
            travel_time = edge_data[0].get('travel_time', 2)
        else:
            travel_time = 2 # Default 2 seconds if not found
            
        cumulative_time += travel_time
        current_timestamp = (start_time + datetime.timedelta(seconds=cumulative_time)).isoformat()
        
        full_path_coords.append({
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": point
            },
            "properties": {
                "time": current_timestamp,
                "icon": "marker",
                "iconstyle": {
                    "color": "red",
                    "fillColor": "red",
                    "fillOpacity": 1,
                    "radius": 5
                }
            }
        })

# 2. Re-create the map
m = folium.Map(location=start_coords, zoom_start=14, tiles="cartodbpositron")

# 3. Add "AntPath" (The moving blue arrows on the road)
# This makes the line itself look alive!
all_coords_for_line = [[G.nodes[node]['y'], G.nodes[node]['x']] for path in lst_paths for node in path]
AntPath(all_coords_for_line, color="#2E86C1", weight=4, delay=1000).add_to(m)

# 4. Add the Moving Driver Marker
TimestampedGeoJson(
    {"type": "FeatureCollection", "features": full_path_coords},
    period="PT1S", # 1 second steps
    add_last_point=True,
    auto_play=True,
    loop=True,
    max_speed=10,
    time_slider_drag_update=True
).add_to(m)

# 5. Add markers for each store stop
for idx, row in dtf_unique.iterrows():
    folium.CircleMarker(
        [row['y'], row['x']], 
        radius=6, 
        color='blue' if row['ID']==0 else 'black', 
        fill=True,
        popup=f"Stop {row['ID']}"
    ).add_to(m)

m.save("animated_driver_route.html")
print("Animation ready! Open 'animated_driver_route.html' and look for the red dot.")
m

Animation ready! Open 'animated_driver_route.html' and look for the red dot.


In [50]:
print(dtf_unique.columns)

Index(['ID', 'City', 'Street Address', 'y', 'x', 'node'], dtype='object')
